In [4]:
!pip install gradio whisper opencv-python numpy yt-dlp pytube moviepy IPython

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 39.8 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of yt-dlp to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 49.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 54.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 58.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 71.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 74.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 65.0 MB/s eta 0:00:00


In [2]:
!pip install gradio whisper opencv-python-headless pytube moviepy google-generativeai

  Using cached gradio-4.37.2-py3-none-any.whl (12.3 MB)
  Using cached whisper-1.1.10.tar.gz (42 kB)
  Preparing metadata (setup.py) ... done
  Using cached pytube-15.0.0-py3-none-any.whl (57 kB)
  Using cached aiofiles-23.2.1-py3-none-any.whl (15 kB)
  Using cached fastapi-0.111.0-py3-none-any.whl (91 kB)
  Using cached ffmpy-0.3.2.tar.gz (5.5 kB)
  Preparing metadata (setup.py) ... done
  Using cached gradio_client-1.0.2-py3-none-any.whl (318 kB)
  Using cached httpx-0.27.0-py3-none-any.whl (75 kB)
  Using cached orjson-3.10.5-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (144 kB)
  Using cached pydub-0.25.1-py2.py3-none-any.whl (32 kB)
  Using cached python_multipart-0.0.9-py3-none-any.whl (22 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [17]:
# Configure Gemini API
GOOGLE_API_KEY = getpass("Enter your Google API key: ")
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
GOOGLE_API_KEY = os.environ.get("GOOGLE_API_KEY")  # Make sure to set this environment variable
genai.configure(api_key=GOOGLE_API_KEY)
gemini_model = genai.GenerativeModel('gemini-pro')

Enter your Google API key: ··········


final output of website

In [18]:
import gradio as gr
import whisper
import cv2
import numpy as np
import os
from pytube import YouTube
from moviepy.editor import VideoFileClip, AudioFileClip
import google.generativeai as genai
import textwrap
from PIL import Image

# Load Whisper model
whisper_model = whisper.load_model("medium")



# Global variable to store captions
global_captions = []

def download_youtube_video(url):
    yt = YouTube(url)
    stream = yt.streams.filter(progressive=True, file_extension='mp4').order_by('resolution').desc().first()
    filename = f"video_{yt.video_id}.mp4"
    stream.download(filename=filename)
    return filename

def extract_audio(video_path):
    audio_path = video_path.replace('.mp4', '.wav')
    video = VideoFileClip(video_path)
    audio = video.audio
    audio.write_audiofile(audio_path)
    return audio_path

def transcribe_segment(audio_segment):
    result = whisper_model.transcribe(audio_segment)
    return result["text"]

def generate_captions(audio_path, segment_duration=5):
    audio = AudioFileClip(audio_path)
    duration = audio.duration
    captions = []

    for start_time in range(0, int(duration), segment_duration):
        end_time = min(start_time + segment_duration, duration)
        segment = audio.subclip(start_time, end_time)
        segment_path = f"temp_segment_{start_time}.wav"
        segment.write_audiofile(segment_path)

        text = transcribe_segment(segment_path)
        captions.append((start_time, end_time, text))

        os.remove(segment_path)

    return captions

def add_caption_to_frame(frame, text):
    h, w = frame.shape[:2]
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 1
    font_thickness = 2
    text_size = cv2.getTextSize(text, font, font_scale, font_thickness)[0]

    text_x = (w - text_size[0]) // 2
    text_y = h - 50

    cv2.rectangle(frame, (text_x - 10, text_y - text_size[1] - 10),
                  (text_x + text_size[0] + 10, text_y + 10),
                  (0, 0, 0), -1)

    cv2.putText(frame, text, (text_x, text_y), font, font_scale, (255, 255, 255), font_thickness)

    return frame

def create_captioned_video(video_path, captions):
    video = VideoFileClip(video_path)

    def add_caption(get_frame, t):
        frame = get_frame(t)
        for start, end, text in captions:
            if start <= t < end:
                return add_caption_to_frame(frame, text)
        return frame

    captioned_video = video.fl(add_caption)

    output_path = video_path.replace('.mp4', '_captioned.mp4')
    captioned_video.write_videofile(output_path, codec='libx264', audio_codec='aac')

    return output_path

def summarize_text(text):
    prompt = f"Please summarize the following text:\n\n{text}\n\nSummary:"
    response = gemini_model.generate_content(prompt)
    return response.text

def process_video(youtube_url, uploaded_video, progress=gr.Progress()):
    global global_captions

    if youtube_url:  # YouTube URL
        progress(0, desc="Downloading video")
        video_path = download_youtube_video(youtube_url)
    elif uploaded_video:  # Uploaded video file
        progress(0, desc="Processing uploaded video")
        video_path = uploaded_video
    else:
        raise ValueError("Please provide either a YouTube URL or upload a video file.")

    progress(0.25, desc="Extracting audio")
    audio_path = extract_audio(video_path)

    progress(0.5, desc="Generating captions")
    global_captions = generate_captions(audio_path)

    progress(0.75, desc="Creating captioned video")
    captioned_video_path = create_captioned_video(video_path, global_captions)

    # Combine all captions into a single text
    full_text = " ".join([text for _, _, text in global_captions])

    progress(0.9, desc="Summarizing content")
    summary = summarize_text(full_text)

    progress(1.0, desc="Done")
    return captioned_video_path, summary

def chat_with_video(question):
    global global_captions

    # Combine all captions into a single text
    full_text = " ".join([text for _, _, text in global_captions])

    prompt = f"""Based on the following video transcript, please answer the question:

Transcript:
{full_text}

Question: {question}

Answer:"""

    response = gemini_model.generate_content(prompt)
    return response.text

# Gradio interface
with gr.Blocks(theme=gr.themes.Default(primary_hue="blue")) as iface:
    gr.Markdown("# Video Captioning, Summarization, and Q&A")

    with gr.Row():
        with gr.Column(scale=2):
            youtube_url = gr.Textbox(label="YouTube URL (optional)")
            uploaded_video = gr.File(label="Upload Video (optional)", type="filepath")
            process_button = gr.Button("Process Video")

        with gr.Column(scale=1):
            chat_input = gr.Textbox(label="Ask a question about the video")
            chat_output = gr.Textbox(label="Answer")
            chat_button = gr.Button("Ask")

    video_output = gr.Video(label="Captioned Video")
    summary_output = gr.Textbox(label="Summary of Video Content")

    process_button.click(
        process_video,
        inputs=[youtube_url, uploaded_video],
        outputs=[video_output, summary_output]
    )

    chat_button.click(
        chat_with_video,
        inputs=[chat_input],
        outputs=[chat_output]
    )

iface.launch(debug=True, share=True)


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://dd275ba8e0f8fcee0c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


MoviePy - Writing audio in video_ry9SYnV3svc.wav


MoviePy - Done.
MoviePy - Writing audio in temp_segment_0.wav


MoviePy - Done.
MoviePy - Writing audio in temp_segment_5.wav


MoviePy - Done.


...

MoviePy - Done.


Moviepy - Building video video_ry9SYnV3svc_captioned.mp4.
MoviePy - Writing audio in video_ry9SYnV3svc_captionedTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video video_ry9SYnV3svc_captioned.mp4



t: 100%|█████████▉| 3197/3198 [00:21<00:00, 174.55it/s, now=None]WARNING:py.warnings:/usr/local/lib/python3.10/dist-packages/moviepy/video/io/ffmpeg_reader.py:123: UserWarning: Warning: in file video_ry9SYnV3svc.mp4, 691200 bytes wanted but 0 bytes read,at frame 3197/3198, at time 133.21/133.21 sec. Using the last valid frame instead.
  warnings.warn("Warning: in file %s, "%(self.filename)+



Moviepy - Done !
Moviepy - video ready video_ry9SYnV3svc_captioned.mp4
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://dd275ba8e0f8fcee0c.gradio.live


In [ ]:
!pip install gradio

In [7]:
! pip install git+https://github.com/openai/whisper.git -q

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
import gradio as gr
import whisper
import cv2
import numpy as np
import os
from pytube import YouTube
from moviepy.editor import *

# Load Whisper model
model = whisper.load_model("medium")

def download_youtube_video(url):
    yt = YouTube(url)
    stream = yt.streams.filter(progressive=True, file_extension='mp4').order_by('resolution').desc().first()
    filename = f"video_{yt.video_id}.mp4"
    stream.download(filename=filename)
    return filename

def extract_audio(video_path):
    audio_path = video_path.replace('.mp4', '.wav')
    video = VideoFileClip(video_path)
    audio = video.audio
    audio.write_audiofile(audio_path)
    return audio_path

def transcribe_segment(audio_segment):
    result = model.transcribe(audio_segment)
    return result["text"]

def generate_captions(audio_path, segment_duration=5):
    audio = AudioFileClip(audio_path)
    duration = audio.duration
    captions = []

    for start_time in range(0, int(duration), segment_duration):
        end_time = min(start_time + segment_duration, duration)
        segment = audio.subclip(start_time, end_time)
        segment_path = f"temp_segment_{start_time}.wav"
        segment.write_audiofile(segment_path)

        text = transcribe_segment(segment_path)
        captions.append((start_time, end_time, text))

        os.remove(segment_path)

    return captions

def add_caption_to_frame(frame, text):
    h, w = frame.shape[:2]
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 1
    font_thickness = 2
    text_size = cv2.getTextSize(text, font, font_scale, font_thickness)[0]

    text_x = (w - text_size[0]) // 2
    text_y = h - 50

    cv2.rectangle(frame, (text_x - 10, text_y - text_size[1] - 10),
                  (text_x + text_size[0] + 10, text_y + 10),
                  (0, 0, 0), -1)

    cv2.putText(frame, text, (text_x, text_y), font, font_scale, (255, 255, 255), font_thickness)

    return frame

def create_captioned_video(video_path, captions):
    cap = cv2.VideoCapture(video_path)
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    output_path = video_path.replace('.mp4', '_captioned.mp4')
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    frame_count = 0
    caption_index = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        current_time = frame_count / fps

        if caption_index < len(captions) and current_time >= captions[caption_index][0]:
            current_caption = captions[caption_index][2]
            if current_time > captions[caption_index][1]:
                caption_index += 1
        else:
            current_caption = ""

        captioned_frame = add_caption_to_frame(frame, current_caption)
        out.write(captioned_frame)

        frame_count += 1

    cap.release()
    out.release()

    return output_path

def process_video(youtube_url, uploaded_video, progress=gr.Progress()):
    if youtube_url:  # YouTube URL
        progress(0, desc="Downloading video")
        video_path = download_youtube_video(youtube_url)
    elif uploaded_video:  # Uploaded video file
        progress(0, desc="Processing uploaded video")
        video_path = uploaded_video
    else:
        raise ValueError("Please provide either a YouTube URL or upload a video file.")

    progress(0.25, desc="Extracting audio")
    audio_path = extract_audio(video_path)

    progress(0.5, desc="Generating captions")
    captions = generate_captions(audio_path)

    progress(0.75, desc="Creating captioned video")
    captioned_video_path = create_captioned_video(video_path, captions)

    progress(1.0, desc="Done")
    return captioned_video_path

iface = gr.Interface(
    fn=process_video,
    inputs=[
        gr.Textbox(label="YouTube URL (optional)"),
        gr.File(label="Upload Video (optional)", type="filepath")
    ],
    outputs=gr.Video(label="Captioned Video"),
    title="Video Captioning with Whisper",
    description="Enter a YouTube URL or upload a video to generate captions using the Whisper model."
)

iface.launch(debug=True, share=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://f78b5b2f03df1dcd53.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


MoviePy - Writing audio in /tmp/gradio/d706781132e4263a54732106b682a96520b73f13/Class 1_toondemy-left-or-right-left-and-right.wav


MoviePy - Done.
MoviePy - Writing audio in temp_segment_0.wav


MoviePy - Done.
MoviePy - Writing audio in temp_segment_5.wav


MoviePy - Done.


...

MoviePy - Done.


MoviePy - Writing audio in temp_segment_115.wav


MoviePy - Done.


  warnings.warn(



Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://f78b5b2f03df1dcd53.gradio.live


In [1]:
import gradio as gr
import whisper
import cv2
import numpy as np
import os
from pytube import YouTube
from moviepy.editor import VideoFileClip, AudioFileClip

# Load Whisper model
model = whisper.load_model("medium")

def download_youtube_video(url):
    yt = YouTube(url)
    stream = yt.streams.filter(progressive=True, file_extension='mp4').order_by('resolution').desc().first()
    filename = f"video_{yt.video_id}.mp4"
    stream.download(filename=filename)
    return filename

def extract_audio(video_path):
    audio_path = video_path.replace('.mp4', '.wav')
    video = VideoFileClip(video_path)
    audio = video.audio
    audio.write_audiofile(audio_path)
    return audio_path

def transcribe_segment(audio_segment):
    result = model.transcribe(audio_segment)
    return result["text"]

def generate_captions(audio_path, segment_duration=5):
    audio = AudioFileClip(audio_path)
    duration = audio.duration
    captions = []

    for start_time in range(0, int(duration), segment_duration):
        end_time = min(start_time + segment_duration, duration)
        segment = audio.subclip(start_time, end_time)
        segment_path = f"temp_segment_{start_time}.wav"
        segment.write_audiofile(segment_path)

        text = transcribe_segment(segment_path)
        captions.append((start_time, end_time, text))

        os.remove(segment_path)

    return captions

def add_caption_to_frame(frame, text):
    h, w = frame.shape[:2]
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 1
    font_thickness = 2
    text_size = cv2.getTextSize(text, font, font_scale, font_thickness)[0]

    text_x = (w - text_size[0]) // 2
    text_y = h - 50

    cv2.rectangle(frame, (text_x - 10, text_y - text_size[1] - 10),
                  (text_x + text_size[0] + 10, text_y + 10),
                  (0, 0, 0), -1)

    cv2.putText(frame, text, (text_x, text_y), font, font_scale, (255, 255, 255), font_thickness)

    return frame

def create_captioned_video(video_path, captions):
    video = VideoFileClip(video_path)

    def add_caption(get_frame, t):
        frame = get_frame(t)
        for start, end, text in captions:
            if start <= t < end:
                return add_caption_to_frame(frame, text)
        return frame

    captioned_video = video.fl(add_caption)

    output_path = video_path.replace('.mp4', '_captioned.mp4')
    captioned_video.write_videofile(output_path, codec='libx264', audio_codec='aac')

    return output_path

def process_video(youtube_url, uploaded_video, progress=gr.Progress()):
    if youtube_url:  # YouTube URL
        progress(0, desc="Downloading video")
        video_path = download_youtube_video(youtube_url)
    elif uploaded_video:  # Uploaded video file
        progress(0, desc="Processing uploaded video")
        video_path = uploaded_video
    else:
        raise ValueError("Please provide either a YouTube URL or upload a video file.")

    progress(0.25, desc="Extracting audio")
    audio_path = extract_audio(video_path)

    progress(0.5, desc="Generating captions")
    captions = generate_captions(audio_path)

    progress(0.75, desc="Creating captioned video")
    captioned_video_path = create_captioned_video(video_path, captions)

    progress(1.0, desc="Done")
    return captioned_video_path

iface = gr.Interface(
    fn=process_video,
    inputs=[
        gr.Textbox(label="YouTube URL (optional)"),
        gr.File(label="Upload Video (optional)", type="filepath")
    ],
    outputs=gr.Video(label="Captioned Video"),
    title="Video Captioning with Whisper",
    description="Enter a YouTube URL or upload a video to generate captions using the Whisper model."
)

iface.launch(debug=True, share=True)

100%|██████████████████████████████████████| 1.42G/1.42G [00:13<00:00, 111MiB/s]


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://fd6838e70031f564b4.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://fd6838e70031f564b4.gradio.live


In [15]:
!pip install -q -U google-generativeai


In [14]:
import os
from getpass import getpass
import textwrap
import google.generativeai as genai
from IPython.display import display
from IPython.display import Markdown

# Set up the API key
GOOGLE_API_KEY = getpass("Enter your Google API key: ")
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

# Install the required package
!pip install -q -U google-generativeai

def to_markdown(text):
    text = text.replace('•', '  *')
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

# Configure the API
genai.configure(api_key=GOOGLE_API_KEY)

# List available models
print("Available models:")
for m in genai.list_models():
    if 'generateContent' in m.supported_generation_methods:
        print(m.name)

# Select a model (you can change this to the specific model you want to use)
model = genai.GenerativeModel('gemini-pro')

# Function to summarize text
def summarize_text(text):
    prompt = f"Please summarize the following text:\n\n{text}\n\nSummary:"
    response = model.generate_content(prompt)
    return response.text

# Get input from the user
large_text = input("Enter or paste your large text content here: ")

# Generate summary
summary = summarize_text(large_text)

# Display the summary
print("\nSummary:")
display(to_markdown(summary))

Enter your Google API key: ··········
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.9/163.9 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 718.3/718.3 kB 11.1 MB/s eta 0:00:00
Available models:
models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-1.5-flash
models/gemini-1.5-flash-001
models/gemini-1.5-flash-latest
models/gemini-1.5-pro
models/gemini-1.5-pro-001
models/gemini-1.5-pro-latest
models/gemini-pro
models/gemini-pro-vision


KeyboardInterrupt: Interrupted by user

In [ ]:
import gradio as gr
import whisper
import cv2
import numpy as np
import os
from pytube import YouTube
from moviepy.editor import VideoFileClip, AudioFileClip
import google.generativeai as genai
import textwrap

# Load Whisper model
whisper_model = whisper.load_model("medium")

# Configure Gemini API
GOOGLE_API_KEY = os.environ.get("GOOGLE_API_KEY")  # Make sure to set this environment variable
genai.configure(api_key=GOOGLE_API_KEY)
gemini_model = genai.GenerativeModel('gemini-pro')

def download_youtube_video(url):
    yt = YouTube(url)
    stream = yt.streams.filter(progressive=True, file_extension='mp4').order_by('resolution').desc().first()
    filename = f"video_{yt.video_id}.mp4"
    stream.download(filename=filename)
    return filename

def extract_audio(video_path):
    audio_path = video_path.replace('.mp4', '.wav')
    video = VideoFileClip(video_path)
    audio = video.audio
    audio.write_audiofile(audio_path)
    return audio_path

def transcribe_segment(audio_segment):
    result = whisper_model.transcribe(audio_segment)
    return result["text"]

def generate_captions(audio_path, segment_duration=5):
    audio = AudioFileClip(audio_path)
    duration = audio.duration
    captions = []

    for start_time in range(0, int(duration), segment_duration):
        end_time = min(start_time + segment_duration, duration)
        segment = audio.subclip(start_time, end_time)
        segment_path = f"temp_segment_{start_time}.wav"
        segment.write_audiofile(segment_path)

        text = transcribe_segment(segment_path)
        captions.append((start_time, end_time, text))

        os.remove(segment_path)

    return captions

def add_caption_to_frame(frame, text):
    h, w = frame.shape[:2]
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 1
    font_thickness = 2
    text_size = cv2.getTextSize(text, font, font_scale, font_thickness)[0]

    text_x = (w - text_size[0]) // 2
    text_y = h - 50

    cv2.rectangle(frame, (text_x - 10, text_y - text_size[1] - 10),
                  (text_x + text_size[0] + 10, text_y + 10),
                  (0, 0, 0), -1)

    cv2.putText(frame, text, (text_x, text_y), font, font_scale, (255, 255, 255), font_thickness)

    return frame

def create_captioned_video(video_path, captions):
    video = VideoFileClip(video_path)

    def add_caption(get_frame, t):
        frame = get_frame(t)
        for start, end, text in captions:
            if start <= t < end:
                return add_caption_to_frame(frame, text)
        return frame

    captioned_video = video.fl(add_caption)

    output_path = video_path.replace('.mp4', '_captioned.mp4')
    captioned_video.write_videofile(output_path, codec='libx264', audio_codec='aac')

    return output_path

def summarize_text(text):
    prompt = f"Please summarize the following text:\n\n{text}\n\nSummary:"
    response = gemini_model.generate_content(prompt)
    return response.text

def process_video(youtube_url, uploaded_video, progress=gr.Progress()):
    if youtube_url:  # YouTube URL
        progress(0, desc="Downloading video")
        video_path = download_youtube_video(youtube_url)
    elif uploaded_video:  # Uploaded video file
        progress(0, desc="Processing uploaded video")
        video_path = uploaded_video
    else:
        raise ValueError("Please provide either a YouTube URL or upload a video file.")

    progress(0.25, desc="Extracting audio")
    audio_path = extract_audio(video_path)

    progress(0.5, desc="Generating captions")
    captions = generate_captions(audio_path)

    progress(0.75, desc="Creating captioned video")
    captioned_video_path = create_captioned_video(video_path, captions)

    # Combine all captions into a single text
    full_text = " ".join([text for _, _, text in captions])

    progress(0.9, desc="Summarizing content")
    summary = summarize_text(full_text)

    progress(1.0, desc="Done")
    return captioned_video_path, summary

iface = gr.Interface(
    fn=process_video,
    inputs=[
        gr.Textbox(label="YouTube URL (optional)"),
        gr.File(label="Upload Video (optional)", type="filepath")
    ],
    outputs=[
        gr.Video(label="Captioned Video"),
        gr.Textbox(label="Summary of Video Content")
    ],
    title="Video Captioning and Summarization",
    description="Enter a YouTube URL or upload a video to generate captions using the Whisper model and summarize the content using Gemini."
)

iface.launch(debug=True, share=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://fdcea371e99d3e2f30.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


MoviePy - Writing audio in /tmp/gradio/d706781132e4263a54732106b682a96520b73f13/Class 1_toondemy-left-or-right-left-and-right.wav


MoviePy - Done.
MoviePy - Writing audio in temp_segment_0.wav


MoviePy - Done.
MoviePy - Writing audio in temp_segment_5.wav


MoviePy - Done.


...

Moviepy - Building video /tmp/gradio/d706781132e4263a54732106b682a96520b73f13/Class 1_toondemy-left-or-right-left-and-right_captioned.mp4.
MoviePy - Writing audio in Class 1_toondemy-left-or-right-left-and-right_captionedTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video /tmp/gradio/d706781132e4263a54732106b682a96520b73f13/Class 1_toondemy-left-or-right-left-and-right_captioned.mp4



t: 100%|█████████▉| 3518/3520 [00:57<00:00, 65.42it/s, now=None]WARNING:py.warnings:/usr/local/lib/python3.10/dist-packages/moviepy/video/io/ffmpeg_reader.py:123: UserWarning: Warning: in file /tmp/gradio/d706781132e4263a54732106b682a96520b73f13/Class 1_toondemy-left-or-right-left-and-right.mp4, 732600 bytes wanted but 0 bytes read,at frame 3519/3520, at time 117.30/117.33 sec. Using the last valid frame instead.
  warnings.warn("Warning: in file %s, "%(self.filename)+



Moviepy - Done !
Moviepy - video ready /tmp/gradio/d706781132e4263a54732106b682a96520b73f13/Class 1_toondemy-left-or-right-left-and-right_captioned.mp4
Keyboard interruption in main thread... closing server.


KeyboardInterrupt: 

In [ ]:
!pip install --upgrade gradio

In [ ]:

import gradio as gr
import whisper
import cv2
import numpy as np
import yt_dlp
import os
import threading
import time
from pytube import YouTube
from moviepy.editor import *
from IPython.display import display, HTML
from deep_translator import GoogleTranslator

# Load Whisper model
model = whisper.load_model("medium")

# List of languages supported by Whisper
LANGUAGES = {
    "en": "English", "zh": "Chinese", "de": "German", "es": "Spanish", "ru": "Russian",
    "ko": "Korean", "fr": "French", "ja": "Japanese", "pt": "Portuguese", "tr": "Turkish",
    "pl": "Polish", "ca": "Catalan", "nl": "Dutch", "ar": "Arabic", "sv": "Swedish",
    "it": "Italian", "id": "Indonesian", "hi": "Hindi", "fi": "Finnish", "vi": "Vietnamese",
    "he": "Hebrew", "uk": "Ukrainian", "el": "Greek", "ms": "Malay", "cs": "Czech",
    "ro": "Romanian", "da": "Danish", "hu": "Hungarian", "ta": "Tamil", "no": "Norwegian",
    "th": "Thai", "ur": "Urdu", "hr": "Croatian", "bg": "Bulgarian", "lt": "Lithuanian",
    "la": "Latin", "mi": "Maori", "ml": "Malayalam", "cy": "Welsh", "sk": "Slovak",
    "te": "Telugu", "fa": "Persian", "lv": "Latvian", "bn": "Bengali", "sr": "Serbian",
    "az": "Azerbaijani", "sl": "Slovenian", "kn": "Kannada", "et": "Estonian", "mk": "Macedonian",
    "br": "Breton", "eu": "Basque", "is": "Icelandic", "hy": "Armenian", "ne": "Nepali",
    "mn": "Mongolian", "bs": "Bosnian", "kk": "Kazakh", "sq": "Albanian", "sw": "Swahili",
    "gl": "Galician", "mr": "Marathi", "pa": "Punjabi", "si": "Sinhala", "km": "Khmer",
    "sn": "Shona", "yo": "Yoruba", "so": "Somali", "af": "Afrikaans", "oc": "Occitan",
    "ka": "Georgian", "be": "Belarusian", "tg": "Tajik", "sd": "Sindhi", "gu": "Gujarati",
    "am": "Amharic", "yi": "Yiddish", "lo": "Lao", "uz": "Uzbek", "fo": "Faroese",
    "ht": "Haitian creole", "ps": "Pashto", "tk": "Turkmen", "nn": "Nynorsk", "mt": "Maltese",
    "sa": "Sanskrit", "lb": "Luxembourgish", "my": "Myanmar", "bo": "Tibetan", "tl": "Tagalog",
    "mg": "Malagasy", "as": "Assamese", "tt": "Tatar", "haw": "Hawaiian", "ln": "Lingala",
    "ha": "Hausa", "ba": "Bashkir", "jw": "Javanese", "su": "Sundanese",
}

def download_youtube_video(url):
    yt = YouTube(url)
    stream = yt.streams.filter(progressive=True, file_extension='mp4').order_by('resolution').desc().first()
    filename = f"video_{yt.video_id}.mp4"
    stream.download(filename=filename)
    return filename

def extract_audio(video_path):
    audio_path = video_path.replace('.mp4', '.wav')
    video = VideoFileClip(video_path)
    audio = video.audio
    audio.write_audiofile(audio_path)
    return audio_path

def transcribe_and_translate(audio_segment, target_language):
    result = model.transcribe(audio_segment)
    text = result["text"]

    if target_language != "en":
        translator = GoogleTranslator(source='auto', target=target_language)
        translated = translator.translate(text)
        return translated
    return text

def generate_captions(audio_path, target_language, segment_duration=5):
    audio = AudioFileClip(audio_path)
    duration = audio.duration
    captions = []

    for start_time in range(0, int(duration), segment_duration):
        end_time = min(start_time + segment_duration, duration)
        segment = audio.subclip(start_time, end_time)
        segment_path = f"temp_segment_{start_time}.wav"
        segment.write_audiofile(segment_path)

        text = transcribe_and_translate(segment_path, target_language)
        captions.append((start_time, end_time, text))

        os.remove(segment_path)

    return captions

def add_caption_to_frame(frame, text):
    h, w = frame.shape[:2]
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 1
    font_thickness = 2
    text_size = cv2.getTextSize(text, font, font_scale, font_thickness)[0]

    # Calculate position to center the text
    text_x = (w - text_size[0]) // 2
    text_y = h - 50  # 50 pixels from the bottom

    # Draw black background rectangle
    cv2.rectangle(frame, (text_x - 10, text_y - text_size[1] - 10),
                  (text_x + text_size[0] + 10, text_y + 10),
                  (0, 0, 0), -1)

    # Draw white text
    cv2.putText(frame, text, (text_x, text_y), font, font_scale, (255, 255, 255), font_thickness)

    return frame

def create_captioned_video(video_path, captions):
    cap = cv2.VideoCapture(video_path)
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    output_path = video_path.replace('.mp4', '_captioned.mp4')
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    frame_count = 0
    caption_index = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        current_time = frame_count / fps

        if caption_index < len(captions) and current_time >= captions[caption_index][0]:
            current_caption = captions[caption_index][2]
            if current_time > captions[caption_index][1]:
                caption_index += 1
        else:
            current_caption = ""

        captioned_frame = add_caption_to_frame(frame, current_caption)
        out.write(captioned_frame)

        frame_count += 1

    cap.release()
    out.release()

    return output_path

def process_youtube_link(youtube_url, target_language, progress=gr.Progress()):
    progress(0, desc="Downloading video")
    video_path = download_youtube_video(youtube_url)

    progress(0.25, desc="Extracting audio")
    audio_path = extract_audio(video_path)

    progress(0.5, desc="Generating captions")
    target_language_code = [code for code, lang in LANGUAGES.items() if lang == target_language][0]
    captions = generate_captions(audio_path, target_language_code)

    progress(0.75, desc="Creating captioned video")
    captioned_video_path = create_captioned_video(video_path, captions)

    progress(1.0, desc="Done")
    return captioned_video_path

iface = gr.Interface(
    fn=process_youtube_link,
    inputs=[
        gr.Textbox(label="YouTube URL"),
        gr.Dropdown(choices=list(LANGUAGES.values()), label="Target Language", value="English")
    ],
    outputs=gr.Video(label="Captioned Video"),
    title="YouTube Video Captioning with Whisper",
    description="Enter a YouTube URL and select a target language to generate a video with captions using the Whisper model."
)

iface.launch(debug=True, share=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://545d119809c3fe228e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


MoviePy - Writing audio in video_ry9SYnV3svc.wav


MoviePy - Done.
MoviePy - Writing audio in temp_segment_0.wav


MoviePy - Done.
MoviePy - Writing audio in temp_segment_5.wav


MoviePy - Done.


...

MoviePy - Done.


MoviePy - Writing audio in temp_segment_130.wav


MoviePy - Done.


  warnings.warn(



Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://545d119809c3fe228e.gradio.live


In [ ]:
!pip install deep-translator

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 2.1 MB/s eta 0:00:00
